In [1]:
import os
import pandas as pd

In [12]:
users_path = "../Impressions.xlsx"
data_path = "2023-05-09T171238.pkl"

In [28]:
users = pd.read_excel(users_path).dropna()
df = pd.read_pickle(data_path)


In [29]:
users

,Name,Username,StartYear,Graduated,Start,End
2,Kelly Aitken,KellyA3,2022,0,2023-01-17,2023-04-23
3,Erika Stark,ErikaS3,2022,0,2022-11-21,2023-04-23
10,Waleed,waleedb,2019,0,2022-11-21,2023-04-23
11,Tubo,TuboS,2019,0,2022-11-21,2023-04-23
12,NickO,NicholaO,2019,0,2022-11-21,2023-04-23
13,Angelo,AngeloD,2019,0,2022-11-21,2023-04-23
14,Benji,benjamir,2019,0,2022-11-21,2023-04-23
15,Michelle,mmarais,2018,0,2022-11-21,2023-04-23
16,Ingrid,ingridm1,2018,0,2022-11-21,2023-04-23
17,Willem Ikink,WillemI,2018,0,2022-11-21,2023-04-23


In [30]:
df.head()

,date,user,modality,descriptor
0,2023-03-27 22:28:05,"Aitken, Kelly (KellyA3)",CT,CT CHEST CTPA
1,2023-03-27 21:07:13,"Aitken, Kelly (KellyA3)",CT,CT CHEST
2,2023-03-27 20:57:33,"Aitken, Kelly (KellyA3)",CT,CT CHEST
3,2023-03-27 20:20:00,"Aitken, Kelly (KellyA3)",CT,CT ABDOMEN
4,2023-03-27 19:32:33,"Aitken, Kelly (KellyA3)",CT,CT CHEST


In [31]:
usernames = df.user.str.extract(r"\((?P<username>.*)\)")
df["username"] = usernames

In [39]:
grouped = df[["username", "modality"]].groupby(["username", "modality"], sort=False)

In [47]:
grouped.size().to_string("output.txt")

In [9]:
summary = df.groupby("user").modality.value_counts()

In [10]:
pd.DataFrame(summary).to_string(fn + ".txt")